In [1]:
import spacy
import numpy as np
from colorama import Fore, Style

In [2]:
def Levenshtein_Distance(str_a,str_b):
    m,n = len(str_a)+1,len(str_b)+1
    matrix = [[i for i in range(n)]for j in range(m)]
    for i in range(m):
        matrix[i][0]=i
    for i in range(1,m):
        for j in range(1,n):
            if str_a[i-1] == str_b[j-1]:
                matrix[i][j] = matrix[i-1][j-1]
            else:
                matrix[i][j] =1+min(matrix[i-1][j-1],matrix[i][j-1],matrix[i-1][j])
    return matrix, matrix[-1][-1]

In [3]:
def changes(matrix):
    m,n = len(matrix)-1,len(matrix[-1])-1
    vectors = [(-1,0),(0,-1),(-1,-1)]
    changes = [[],[],[]]
    while m > 0 or n > 0:
        best = matrix[m][n]
        cat = None
        chg = (-1,-1)
        for category,(i,j) in enumerate(vectors):
            if matrix[m+i][n+j] <= best:
                best = matrix[m+i][n+j]
                cat = category
                chg = (m+i, n+j)
        if best != matrix[m][n]:
            changes[cat].append(chg)
        m = chg[0]
        n =chg[1]
    return(changes)

In [4]:
def list_changes(m,str_a,str_b):
    chg = changes(m)
    a_tab = list(str_a)
    for j in chg[0]:
        print("".join(a_tab),end =" -> ")
        a_tab[j[0]]="_"
        print("".join(a_tab))
        a_tab.pop(j[0])
    for j in chg[1]:
        print("".join(a_tab),end =" -> ")
        print("".join(a_tab[:j[0]])+"*"+str_b[j[1]]+"*"+"".join(a_tab[j[0]:]))
        a_tab.insert(j[0],str_b[j[1]])
    for j in chg[2]:
        print("".join(a_tab),end =" -> ")
        tmp_tab = a_tab
        tmp_tab[j[1]] = "|"+a_tab[j[1]]+" -> "+str_b[j[1]]+"|"
        print("".join(tmp_tab))
        a_tab[j[1]] = str_b[j[1]]
    print("".join(a_tab))

In [5]:
strings = [("los", "kloc"),("Łódź", "Lodz"),("kwintesencja", "quintessence"),
           ("ATGAATCTTACCGCCTCG", "ATGAGGCTCTGGCCCCTG"),("Kraków, ulica Kawiory", "Czchów, ulica Sezamkowa")]
for str_a,str_b in strings:
    print(str_a,"\t-\t",str_b)
    matrix,distance = Levenshtein_Distance(str_a, str_b)
    print(distance)
    list_changes(matrix,str_a,str_b)

los 	-	 kloc
2
los -> *k*los
klos -> klo|s -> c|
kloc
Łódź 	-	 Lodz
3
Łódź -> Łód|ź -> z|
Łódz -> Ł|ó -> o|dz
Łodz -> |Ł -> L|odz
Lodz
kwintesencja 	-	 quintessence
5
kwintesencja -> kwintesenc_a
kwintesenca -> kwintes*s*enca
kwintessenca -> kwintessenc|a -> e|
kwintessence -> k|w -> u|intessence
kuintessence -> |k -> q|uintessence
quintessence
ATGAATCTTACCGCCTCG 	-	 ATGAGGCTCTGGCCCCTG
7
ATGAATCTTACCGCCTCG -> ATGAATCTTACCGCCT_G
ATGAATCTTACCGCCTG -> ATGAATCT*C*TACCGCCTG
ATGAATCTCTACCGCCTG -> ATGAATCTCTACC|G -> C|CCTG
ATGAATCTCTACCCCCTG -> ATGAATCTCTA|C -> G|CCCCTG
ATGAATCTCTAGCCCCTG -> ATGAATCTCT|A -> G|GCCCCTG
ATGAATCTCTGGCCCCTG -> ATGAA|T -> G|CTCTGGCCCCTG
ATGAAGCTCTGGCCCCTG -> ATGA|A -> G|GCTCTGGCCCCTG
ATGAGGCTCTGGCCCCTG
Kraków, ulica Kawiory 	-	 Czchów, ulica Sezamkowa
11
Kraków, ulica Kawiory -> Kraków, ulica *e*Kawiory
Kraków, ulica eKawiory -> Kraków, ulica *S*eKawiory
Kraków, ulica SeKawiory -> Kraków, ulica SeKawior|y -> a|
Kraków, ulica SeKawiora -> Kraków, ulica SeKawio|r -> 

In [6]:
def LCS(str_a,str_b):
    m,n = len(str_a)+1,len(str_b)+1
    matrix = [[0 for i in range(n)]for j in range(m)]
    for i in range(1,m):
        for j in range(1,n):
            if str_a[i-1] == str_b[j-1]:
                matrix[i][j] = matrix[i-1][j-1]+1
            else:
                matrix[i][j] =max(matrix[i-1][j-1],matrix[i][j-1],matrix[i-1][j])
    return matrix, matrix[-1][-1]

In [7]:
str_a,str_b= "Kraków, Kawiory", "Czchów, Basztowa"
m,longest = LCS(str_a,str_b)

In [8]:
nlp = spacy.load('pl_core_news_lg') 

In [9]:
with open('romeo-i-julia-700.txt','r',encoding = 'utf-8') as file:
    content = file.read()
tokens = np.array(nlp(content))

In [10]:
for token in tokens[:15]:
    print('{:<15}{:<12}'.format(token.text, token.pos_))

William        PROPN       
Shakespeare    PROPN       


             SPACE       
Romeo          PROPN       
i              CCONJ       
Julia          PROPN       

              SPACE       
tłum           NOUN        
.              PUNCT       
Józef          PROPN       
Paszkowski     PROPN       


             SPACE       
ISBN           PROPN       
978            X           
-              PUNCT       


In [11]:
def remove_from2(tokens):
    n = len(tokens)
    size = int(0.97*n)
    signs1 = set(np.random.choice(range(0,n),size =(size,),replace=False))
    signs2 = set(np.random.choice(range(0,n),size =(size,),replace=False))
    text1 =[tokens[i] for i in range(n) if i in signs1 ]
    text2 =[tokens[i] for i in range(n) if i in signs2 ]
    return text1,text2

In [12]:
tokens1,tokens2 = remove_from2(tokens)

In [13]:
m,longest = LCS([str(token) for token in tokens1],[str(token) for token in tokens2])

In [14]:
longest

2540

In [15]:
def LCS_changes(str_a,str_b):
    m,n = len(str_a)+1,len(str_b)+1
    matrix = [[(0,0) for i in range(n)]for j in range(m)]
    vectors = [-1,-1],[0,-1],[-1,0]
    for i in range(1,m):
        for j in range(1,n):
            if str_a[i-1] == str_b[j-1]:
                matrix[i][j] = matrix[i-1][j-1][0]+1,3
            else:
                best = matrix[i][j]
                if matrix[i][j-1][0] >= best[0]:
                    best = matrix[i][j-1][0],0
                if matrix[i-1][j][0] >= best[0]:
                    best = matrix[i-1][j][0],1
                if matrix[i-1][j-1][0] >= best[0]:
                    best = matrix[i-1][j-1][0],2
                matrix[i][j] = best
    i, j = m-1, n-1
    result = []
    while i>-1 and j>-1:
        _, x =matrix[i][j]
        if x == 0:
            result+=[("<"+str_a[i-1]+'>-pierwszy: '+str(i-1))]
            j -= 1
        elif x == 1:
            result+=[("<"+str_b[j-1]+'>-drugi: '+str(j-1))]
            i -= 1
        elif x == 2:
            result+=[("<"+str_a[i-1]+">-pierwszy: "+str(i-1)+"\t<"+str_b[j-1]+">-drugi: "+str(j-1))]
            i -= 1
            j -= 1
        elif x == 3:
            result+=[("<"+str_a[i-1]+'>-oba: '+str(i-1)+", "+str(j-1))]
            i -= 1
            j -= 1
    n = len(result)
    array = [result[i] for i in range(n-1,-1,-1)]
    for a in array:
        print(a)

In [16]:
with open("tokens1.txt","w") as file:
    file.write("".join([str(t)+" " for t in tokens1]))
with open("tokens2.txt","w") as file:
    file.write("".join([str(t)+" " for t in tokens2]))

In [17]:
def compare(file_path1,file_path2):
    lines1,lines2 = [],[]
    with open("tokens1.txt") as f1, open("tokens2.txt") as f2:
        for x in f1:
            lines1.append(x.rstrip())
        for y in f2:
            lines2.append(y.rstrip())
    print(LCS_changes(lines1,lines2))

In [18]:
compare("tokens1.txt","tokens2.txt")

<>-pierwszy: -1
<William Shakespeare>-oba: 0, 0
<>-oba: 1, 1
< Romeo i Julia>-pierwszy: 2	< i Julia>-drugi: 2
< tłum . Józef Paszkowski>-oba: 3, 3
<>-oba: 4, 4
< ISBN 978 - 83 - 288 - 2903 - 9>-oba: 5, 5
<>-oba: 6, 6
<>-oba: 7, 7
<>-oba: 8, 8
< OSOBY :>-oba: 9, 9
<  * ESKALUS — panujący w Weronie>-pierwszy: 10	<  * ESKALUS — książę panujący w Weronie>-drugi: 10
<  * PARYS — młody Weroneńczyk szlachetnego rodu , krewny księcia>-pierwszy: 11	<  * PARYS — młody Weroneńczyk szlachetnego rodu krewny>-drugi: 11
<  * MONTEKI , KAPULET — naczelnicy dwóch domów nieprzyjaznych sobie>-oba: 12, 12
<  * STARZEC — stryjeczny brat Kapuleta>-oba: 13, 13
<  * ROMEO syn Montekiego>-pierwszy: 14	<  * ROMEO — syn Montekiego>-drugi: 14
<  * MERKUCJO — krewny księcia>-oba: 15, 15
<  BENWOLIO — synowiec Montekiego>-pierwszy: 16	<  * BENWOLIO — synowiec Montekiego>-drugi: 16
<  * TYBALT — krewny Pani Kapulet>-pierwszy: 17	<  TYBALT — krewny Pani Kapulet>-drugi: 17
<  * LAURENTY — ojciec franciszkanin>-oba: 18